In [2]:
import redframes as rf
import pandas as pd
import numpy as np

In [3]:
import redframes as rf

df = rf.load("nfl.csv")

(
    df.filter(lambda row: (
        # row["Team"].isin(["BUF", "KCC"]) & 
        (row["Points"] >= 100)
    ))
    .combine(["First Name", "Last Name"], sep=" ", into="name")
    .rename({"Points": "points", "Team": "team", "Bye Week": "bye", "Position": "position"}) # reverse this?
    .select(["name", "position", "team", "bye", "points"])
    .sort(["points", "position"], reverse=True)
    .group("position")
    .take(2)
)

,name,position,team,bye,points
0,Josh Allen,QB,BUF,7,427.8
1,Patrick Mahomes,QB,KCC,8,406.6
2,Derrick Henry,RB,TEN,6,305.8
3,Jonathan Taylor,RB,IND,14,303.5
4,Cooper Kupp,WR,LAR,7,254.3
5,Justin Jefferson,WR,MIN,7,226.3
6,Travis Kelce,TE,KCC,8,213.8
7,Dallas Cowboys,D,DAL,9,165.3
8,Mark Andrews,TE,BAL,10,164.0
9,Daniel Carlson,K,LVR,6,158.1


In [11]:
df = rf.load("nfl.csv")

(
    df.filter(lambda row: (
        row["Team"].isin(["BUF", "KCC"]) & 
        (row["Points"] >= 100)
    ))
    .combine(["First Name", "Last Name"], sep=" ", into="name")
    .rename({"Points": "points", "Team": "team", "Bye Week": "bye"}) # reverse this?
    .select(["name", "team", "bye", "points"])
    .sort(["points"], reverse=True)
    .accumulate("points", method="sum", into="cum_sum_points")
)

,name,team,bye,points,cum_sum_points
0,Josh Allen,BUF,7,427.8,427.8
1,Patrick Mahomes,KCC,8,406.6,834.4
2,Travis Kelce,KCC,8,213.8,1048.2
3,Devin Singletary,BUF,7,192.3,1240.5
4,Stefon Diggs,BUF,7,183.1,1423.6
5,Clyde Edwards-Helaire,KCC,8,172.0,1595.6
6,Gabriel Davis,BUF,7,168.5,1764.1
7,JuJu Smith-Schuster,KCC,8,154.0,1918.1
8,Harrison Butker,KCC,8,143.5,2061.6
9,Tyler Bass,BUF,7,141.8,2203.4


In [2]:
df = pd.read_csv("nfl.csv")

df.groupby("Team")

def aggregate(df, apropos, by=None):
    data = self._data.copy()
    if by:
        data = data.groupby(by)
    data = data.agg(**apropos).reset_index()
    return DataFrame(data)


In [ ]:
df.assign(cum_average=df.groupby('Team', sort=False)['Points'].transform(lambda x: x.expanding().mean()))

In [ ]:
rf.export(df, "example.csv")
df.save("")

In [26]:
# group by
# take
# mutate
# cum sum
# aggregate

from typing import Literal

# df.sort_values("Points", ascending=False).groupby(["Position"]).head(4).cummean()

df = pd.read_csv("nfl.csv")
# df = df.drop("Rank", axis=1)
# df = df.sort_values("Points", ascending=True)
# df["rank"] = df["Points"].rank(method="first")
# df


def rank(df: pd.DataFrame, column: str, method: Literal["average", "min", "max", "first", "dense"], into: str, reverse: bool = False) -> pd.DataFrame:
    df = df.copy()
    df[into] = df[column].rank(method=method, ascending=not reverse)
    return df

rank(df, "Points", method="dense", into="rank", reverse=True).sort_values("rank")


,Rank,ID,Last Name,First Name,Position,Team,Points,VBD,Bye Week,rank
31,32,13589,Allen,Josh,QB,BUF,427.8,248.2,7,1.0
39,40,13116,Mahomes,Patrick,QB,KCC,406.6,229.1,8,2.0
46,47,14779,Herbert,Justin,QB,LAC,389.5,213.7,8,3.0
54,55,7836,Rodgers,Aaron,QB,GBP,358.2,195.8,14,4.0
57,58,14056,Murray,Kyler,QB,ARI,353.1,191.0,13,5.0
...,...,...,...,...,...,...,...,...,...,...
461,491,15054,Ricci,Giovanni,RB,CAR,-0.2,47.3,13,441.0
412,429,15777,Austin III,Calvin,WR,PIT,-0.3,57.4,9,442.0
462,492,15480,Funk,Jake,RB,LAR,-0.3,47.3,7,442.0
532,590,15694,Willis,Malik,QB,TEN,-0.8,-107.0,6,443.0


In [28]:

import redframes as rf
import pandas as pd
import numpy as np

df = pd.read_csv("nfl.csv")

apropos = {
    "count": ("Points", rf.compute.count),
    "sum": ("Points", rf.compute.sum),
    "mean": ("Points", rf.compute.mean),
    "median": ("Points", rf.compute.median), 
    "min": ("Points", rf.compute.min), 
    "max": ("Points", rf.compute.max),
    "std": ("Points", rf.compute.std)
}
# df.groupby(["Team", "Position"]).agg(**apropos).reset_index()
(
    df.agg(**{
        "mean_points": ("Points", np.mean), 
        "sum_points": ("Points", np.sum), 
        "min_bye": ("Bye Week", np.min)
    })
    .T
    .reset_index(drop=True)
    .fillna(method="bfill")
    .fillna(method="ffill")
    .head(1)
)


,mean_points,sum_points,min_bye
0,82.421536,44013.1,6.0


In [ ]:
from __future__ import annotations

from typing import Any, Callable
import pandas as pd
import pandas.core.groupby.generic as pg

def aggregate(df: pd.DataFrame | pg.DataFrameGroupBy, aggregations: dict[str, tuple[str, Callable[..., Any]]]) -> pd.DataFrame:
    if not isinstance(aggregations, dict):
        raise TypeError("aggregations type is invalid, must be dict[str, tuple[str, Callable[..., Any]]]")
    if isinstance(df, pg.DataFrameGroupBy):
        return df.agg(**apropos).reset_index()
    df = df.agg(**aggregations)
    df = df.T
    df = df.reset_index(drop=True)
    df = df.fillna(method="bfill")
    df = df.fillna(method="ffill")
    df = df.head(1)
    return df
